## RMSE predicted scaled vs RMSE predicted

From what was observed in the prior notebook, we recognized that values predicted did not tend on the same scale as target values.  It was thought perhaps scaling those predicted values (to match the same scale) would improve RMSE but oddly enough it did not.

Now we seek to better understand the relationship of target values vs predicted and scaled predicted and get to the bottom of this

In [1]:
# import what we will be using
from __future__ import print_function


import pandas as pd
import numpy as np

# load the data and take a quick peek - we will use the beer_style summary we created previously
beer_styles = pd.read_csv("../data/beer_style_summary.csv")
beer_styles.shape

(104, 19)

In [2]:
# so 104 rows... let's build 2 separate lists to hold unique beer styles for test and train data

# start by generating 10 random ints 
aryRand = np.random.randint(104, size=10)

# next... convert this array to a series... take head/tail to form our actual indexes of beer style
srRand = pd.Series(aryRand)
dfTop5 = srRand.head().to_frame()
# hack a dummy column
dfTop5['dummy'] = ""
dfTop5.set_index(0, inplace=True)
# merge to get 5 random beer_styles
dfTop5 = dfTop5.merge(beer_styles, how="inner", left_index=True, right_index=True)

# do it all over for train
dfBottom5 = srRand.tail().to_frame()
# hack a dummy column
dfBottom5['dummy'] = ""
dfBottom5.set_index(0, inplace=True)
# merge to get 5 random beer_styles
dfBottom5 = dfBottom5.merge(beer_styles, how="inner", left_index=True, right_index=True)

print ('dfTop5\n', dfTop5['beer_style'])
print ('\n')
print ('dfBottom5\n', dfBottom5['beer_style'])

dfTop5
 11    American Double / Imperial Stout
36                      Czech Pilsener
42                  English Barleywine
74                      Lambic - Fruit
14             American Pale Ale (APA)
Name: beer_style, dtype: object


dfBottom5
 68                Irish Red Ale
38    Dortmunder / Export Lager
57           Flanders Oud Bruin
93                  Schwarzbier
30                  Black & Tan
Name: beer_style, dtype: object


In [3]:
# load all beer reviews
beer_reviews = pd.read_csv("../data/beer_reviews.csv")
beer_reviews.shape

(1586614, 13)

In [4]:
# first for test
dfTest = dfTop5.merge(beer_reviews, on="beer_style")
dfTest.shape

(151595, 32)

In [5]:
'''
want to keep row count to a reasonable number - would not do this in a real life scenario but
for purposes of what is happening here... being able to run this stuff quickly allows
more time for the important stuff -- actual coding and learning
'''
def trimDataRows(df, rowcount):
    srRand = pd.Series(np.random.randint(df.shape[0], size=rowcount))
    dfMerge = srRand.to_frame()

    # hack a dummy column
    dfMerge['dummy'] = ""
    dfMerge.set_index(0, inplace=True)

    # merge to get 5 random beer_styles
    df = dfMerge.merge(df, how="inner", left_index=True, right_index=True)
    # drop dummy
    try:
        df.drop('dummy_x', axis=1, inplace=True)
        df.drop('dummy_y', axis=1, inplace=True)
    except:
        pass

    return df

In [6]:
dfTest = trimDataRows(dfTest, 25000)
dfTest.head()

,beer_style,number_of_distinct_reviewers,number_of_distinct_brewerys,number_of_distinct_beer,total_brewerys,pct_brewerys,mean_distinct_brewerys,sd_distinct_brewerys,Z_distinct_brewerys,total_reviewers,...,review_time,review_overall,review_aroma,review_appearance,review_profilename,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,American Double / Imperial Stout,8049,470,874,5742,0.081853,411.259615,346.078418,0.169731,33387,...,1212201268,4.5,4.5,4.0,grumpy,4.0,4.5,Imperial Stout,NaN,42964
3,American Double / Imperial Stout,8049,470,874,5742,0.081853,411.259615,346.078418,0.169731,33387,...,1278811077,4.5,3.5,3.5,4000qtrap,3.0,2.5,Leviathan Imperial Stout,10.0,39606
11,American Double / Imperial Stout,8049,470,874,5742,0.081853,411.259615,346.078418,0.169731,33387,...,1262208676,3.0,3.5,4.0,JamesS,3.0,3.0,2001 - A Stout Odyssey,8.0,55096
16,American Double / Imperial Stout,8049,470,874,5742,0.081853,411.259615,346.078418,0.169731,33387,...,1309119785,5.0,4.5,4.5,UMNbeerhunter,5.0,4.5,The Worthy Adversary,9.2,57172
19,American Double / Imperial Stout,8049,470,874,5742,0.081853,411.259615,346.078418,0.169731,33387,...,1305386559,4.5,4.0,5.0,deadbody,3.0,4.5,The Worthy Adversary,9.2,57172


In [7]:
# pick our feature/predictor list(s)
X_variable = list(['review_appearance','review_taste','review_aroma'])
y_variable = list(['review_overall'])

print('X_variable:\t', X_variable)
print('y_variable:\t', y_variable)

X_variable:	 ['review_appearance', 'review_taste', 'review_aroma']
y_variable:	 ['review_overall']


In [47]:
# going one step further now... we create y_target as a dataset of just target values, next X_source
y_target = dfTest[y_variable]
y_target.columns = ['target_value']
y_target.head()

,target_value
0,4.5
3,4.5
11,3.0
16,5.0
19,4.5


In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

# visualization
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [46]:
# next we need to get y_predicted
# this is the prediction (DecisionTree max_depth=6)
decision_tree = DecisionTreeRegressor(max_depth=6)
decision_tree.fit(dfTest[X_variable], dfTest[y_variable])
y_predicted = pd.DataFrame(decision_tree.predict(dfTest[X_variable]))

y_predicted.columns = ['predicted_value']
y_predicted.head()

,predicted_value
0,4.318824
1,2.824607
2,3.251482
3,4.428309
4,4.311594


Create a new dataframe (dfResults) - merge together y_predicted and y_target, then add a final column (predicted_value_scaled) this new column will contain our scaled values


In [49]:
# define a function to scale predicted results to .5 intervals
def scale_to_nearest_half_int(x):
    return round(x * 2) / 2

In [48]:
dfResults = pd.merge(y_target, y_predicted, how="inner", left_index=True, right_index=True)
dfResults.head()  

,target_value,predicted_value
0,4.5,4.318824
3,4.5,4.428309
11,3.0,3.695111
16,5.0,4.071133
19,4.5,4.020777


In [52]:
#create predicted_value_scaled
dfResults['predicted_value_scaled'] = dfResults.apply(lambda x: scale_to_nearest_half_int(x['predicted_value']), axis=1)
dfResults.head(25)

,target_value,predicted_value,predicted_value_scaled
0,4.5,4.318824,4.5
3,4.5,4.428309,4.5
11,3.0,3.695111,3.5
16,5.0,4.071133,4.0
19,4.5,4.020777,4.0
20,4.0,4.020777,4.0
25,4.0,3.724275,3.5
25,4.0,3.724275,3.5
30,3.0,4.114924,4.0
31,4.0,4.071133,4.0


In [51]:
# save this off as csv... can peek this over in excel before going further
dfResults.to_csv("../data/rmse_analysis.csv")

exporting to excel was helpful.... a few things I discovered glancing through the data

* Most predicted values did NOT match target values
* Scaling did in fact match far more numbers however at a cost
    * where predicted value was rounded down - in many instances it now moved further away from the target
    * where predicted value was rounded up - in many instances if still moved further away from the target

Next thing that would be helpful in understanding this would be to build some counts
* Count of predicted_value matching target_value
* Count of predicted_value_scaled matching target_value
* Count of predicted_value_scaled now further away than predicted_value from target_value


In [60]:
# calc some new columns now
#---------------------------------

# exact matches (1/0)
dfResults['target_predicted_match'] = dfResults.apply(lambda x: 1 if x['predicted_value'] == x['target_value'] else 0, axis=1)
dfResults['target_scaled_match'] = dfResults.apply(lambda x: 1 if x['predicted_value_scaled'] == x['target_value'] else 0, axis=1)

# actual diff
dfResults['target_predicted_diff'] = dfResults.apply(lambda x: x['target_value'] - x['predicted_value'], axis=1)
dfResults['target_scaled_diff'] = dfResults.apply(lambda x: x['target_value'] - x['predicted_value_scaled'], axis=1)

# ABS(diff)
dfResults['target_predicted_abs_diff'] = dfResults.apply(lambda x: np.absolute(x['target_predicted_diff']), axis=1)
dfResults['target_scaled_abs_diff'] = dfResults.apply(lambda x: np.absolute(x['target_scaled_diff']), axis=1)

dfResults.head(25)

,target_value,predicted_value,predicted_value_scaled,target_predicted_match,target_scaled_match,target_predicted_diff,target_scaled_diff,target_predicted_abs_diff,target_scaled_abs_diff
0,4.5,4.318824,4.5,0,1,0.181176,0.0,0.181176,0.0
3,4.5,4.428309,4.5,0,1,0.071691,0.0,0.071691,0.0
11,3.0,3.695111,3.5,0,0,-0.695111,-0.5,0.695111,0.5
16,5.0,4.071133,4.0,0,0,0.928867,1.0,0.928867,1.0
19,4.5,4.020777,4.0,0,0,0.479223,0.5,0.479223,0.5
20,4.0,4.020777,4.0,0,1,-0.020777,0.0,0.020777,0.0
25,4.0,3.724275,3.5,0,0,0.275725,0.5,0.275725,0.5
25,4.0,3.724275,3.5,0,0,0.275725,0.5,0.275725,0.5
30,3.0,4.114924,4.0,0,0,-1.114924,-1.0,1.114924,1.0
31,4.0,4.071133,4.0,0,1,-0.071133,0.0,0.071133,0.0


In [100]:
# Now return some aggregates
dfAgg = pd.DataFrame(
    dfResults.agg({
       'target_predicted_match': 'sum',
       'target_scaled_match': 'sum',
       'target_predicted_diff': 'sum',
       'target_scaled_diff': 'sum',
       'target_predicted_abs_diff': 'sum',
       'target_scaled_abs_diff': 'sum'
    })
    )
dfAgg.columns = ['value']
dfAgg.reset_index(inplace=True)
dfAgg = dfAgg.pivot_table(columns='index', values='value')

dfAgg.rename(columns={
                        'target_predicted_match' : 'total_matches_predicted',
                        'target_scaled_match' : 'total_matches_scaled',
                        'target_predicted_diff' : 'sum_predicted_diff',
                        'target_scaled_diff' : 'sum_scaled_diff',
                        'target_predicted_abs_diff' : 'sum_predicted_abs_diff',
                        'target_scaled_abs_diff' : 'sum_scaled_abs_diff'                   
                      }, inplace=True)
dfAgg

index,sum_predicted_abs_diff,sum_predicted_diff,total_matches_predicted,sum_scaled_abs_diff,sum_scaled_diff,total_matches_scaled
value,2518.754579,869.503983,0.0,2560.5,932.5,1032.0


So this is quite revealing

As expected we have ZERO direct matches between target and predicted values
We have 1032 matches between scaled and predicted... but that is a small percentage against the 25,000 values we predicted

Both abs diff and actual diff show better results for the predicted values over the scaled values
to be clear... predicted of 869 is lower than scaled of 932.5 and likewise with abs predicted of 2518 is also lower than scaled abs of 2560

Where does that leave us

Ultimately... we predicted 0 actual values and even after scaling we predicted (1032 / 25000) or 4.12%

That's awful... perhaps the next best thing to do is ReConsider how this question was approached to begin with.  That is... approach this question as a classification question not a linear question and that is the function of the last notebook in this project